# Binding detailed example: Miniut2

Let's try a non-trivial example: Minuit2 (6.14.0 standalone edition)

### Requirements

* Minuit2 6.14.0 standalone edition
* PyBind11
* NumPy
* C++11 compatible compiler

### Expectations
* Be able to minimize a very simple function and get some parameters

# Step 1: get source

* Download Minuit2 source (provided in `Minuit2-6.14.0-Source`)
* Install PyBind11 or add as submodule (assuming install for now)
* Setup build of Minuit2 (use provided `setup.py`)
    - Simplified assumption: pybind11 already installed
    - Uses Python 3 (or install pathlib2 on Python2)
    - I left the Windows code in but use at own risk

# Step 2: plan interface

* You should know what the C++ looks like, and know what you want the Python to look like

In [79]:
%%writefile simpleminuit.cpp

#include <Minuit2/FCNBase.h>
#include <Minuit2/FunctionMinimum.h>
#include <Minuit2/MnPrint.h>
#include <Minuit2/MnMigrad.h>

using namespace ROOT::Minuit2;

class SimpleFCN : public FCNBase {
    double Up() const override {return 0.5;}
    double operator()(const std::vector<double> &v) const override {
        std::cout << "val = " << v.at(0) << std::endl;
        return v.at(0)*v.at(0);
    }
};

int main() {
    SimpleFCN fcn;
    MnUserParameters upar;
    upar.Add("x", 1., 0.1);
    MnMigrad migrad(fcn, upar);
    FunctionMinimum min = migrad();
    std::cout << min << std::endl;
}

Overwriting simpleminuit.cpp


In [80]:
%%writefile CMakeLists.txt
cmake_minimum_required(VERSION 3.11)
project(Minuit2SimpleExamle LANGUAGES CXX)

add_subdirectory(Minuit2-6.14.0-Source)
add_executable(simpleminuit simpleminuit.cpp)
target_link_libraries(simpleminuit PRIVATE Minuit2::Minuit2)

Overwriting CMakeLists.txt


In [81]:
!cmake .

-- Configuring done
-- Generating done
-- Build files have been written to: /Users/henryiii/git/presentations/physics/pychep


In [82]:
!cmake --build .

[  3%] Built target Math
[ 97%] Built target Minuit2
Scanning dependencies of target simpleminuit
[ 98%] Building CXX object CMakeFiles/simpleminuit.dir/simpleminuit.cpp.o
[100%] Linking CXX executable simpleminuit
[100%] Built target simpleminuit


In [83]:
!./simpleminuit

val = 1
val = 1.001
val = 0.999
val = 1.0006
val = 0.999402
val = -8.23008e-11
val = 0.000345267
val = -0.000345267
val = -8.23008e-11
val = 0.000345267
val = -0.000345267
val = 6.90533e-05
val = -6.90535e-05

Minuit did successfully converge.
# of function calls: 13
minimum function Value: 6.773427082119e-21
minimum edm: 6.773427081817e-21
minimum internal state vector: LAVector parameters:
 -8.230083281546e-11

minimum internal covariance matrix: LASymMatrix parameters:
              1


# ext. ||   Name    ||   type  ||     Value     ||  Error +/- 

   0   ||         x ||  free   || -8.230083281546e-11 ||0.7071067811865







## what we need

* subclass FCNBase
* MnUserParameters (constructor and Add)
* MnMigrad (constructor and operator())
* FunctionMinimum (cout)


In [33]:
%%writefile setup.py

from setuptools import setup, Extension
from setuptools.command.build_ext import build_ext
import sys
import setuptools
from pathlib import Path # Python 3 or Python 2 backport: pathlib2
import pybind11 # Real code should defer this import

sources = set(str(p) for p in Path('Minuit2-6.14.0-Source/src').glob('**/*.cxx'))
sources.remove('Minuit2-6.14.0-Source/src/TMinuit2TraceObject.cxx')

## Add your sources to `sources`
sources |= set(str(p) for p in Path('pyminuit2').glob('*.cpp'))

ext_modules = [
    Extension(
        'minuit2',
        list(sources),
        include_dirs=[
            pybind11.get_include(False),
            pybind11.get_include(True),
            'Minuit2-6.14.0-Source/inc',
        ],
        language='c++',
        define_macros=[('WARNINGMSG', None),
                       ('MATH_NO_PLUGIN_MANAGER', None),
                       ('ROOT_Math_VecTypes', None)
                      ],
    ),
]

class BuildExt(build_ext):
    """A custom build extension for adding compiler-specific options."""
    c_opts = {
        'msvc': ['/EHsc'],
        'unix': [],
    }

    if sys.platform == 'darwin':
        c_opts['unix'] += ['-stdlib=libc++', '-mmacosx-version-min=10.7']

    def build_extensions(self):
        ct = self.compiler.compiler_type
        opts = self.c_opts.get(ct, [])
        if ct == 'unix':
            opts.append('-DVERSION_INFO="%s"' % self.distribution.get_version())
            opts.append('-std=c++14')
            opts.append('-fvisibility=hidden')
        elif ct == 'msvc':
            opts.append('/DVERSION_INFO=\\"%s\\"' % self.distribution.get_version())
        for ext in self.extensions:
            ext.extra_compile_args = opts
        build_ext.build_extensions(self)

setup(
    name='minuit2',
    version='6.14.0',
    author='Henry Schriener',
    author_email='hschrein@cern.ch',
    url='https://github.com/GooFit/Minuit2',
    description='A Pybind11 Minuit2 binding',
    long_description='',
    ext_modules=ext_modules,
    install_requires=['pybind11>=2.2', 'numpy>=1.10'],
    cmdclass={'build_ext': BuildExt},
    zip_safe=False,
)

Overwriting setup.py


In [35]:
!python setup.py build_ext

running build_ext
building 'minuit2' extension
creating build/temp.macosx-10.7-x86_64-3.6
creating build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source
creating build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src
creating build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/math
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/MnUserParameters.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/MnUserParameters.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreacha

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/AnalyticalGradientCalculator.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/AnalyticalGradientCalculator.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henry

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/FumiliBuilder.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/FumiliBuilder.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m 

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/FumiliStandardMaximumLikelihoodFCN.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/FumiliStandardMaximumLikelihoodFCN.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/MnFcn.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/MnFcn.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/FumiliGradientCalculator.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/FumiliGradientCalculator.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.loc

## Structure of a PyBind11 program

#### main.cpp

* Builds module
* Avoids imports (fast compile)

```cpp
include <pybind11/pybind11.h>
namespace py = pybind11;

void init_part1(py::module &);
void init_part2(py::module &);

PYBIND11_MODULE(mymodule, m) {
    m.doc() = "Real code would never have such poor documentation...";
    
    init_part1(m);
    init_part2(m);
}
```

In [38]:
mkdir pyminuit2

In [87]:
%%writefile pyminuit2/pyminuit2.cpp

#include <pybind11/pybind11.h>

namespace py = pybind11;

void init_FCNBase(py::module &);
void init_MnUserParameters(py::module &);
void init_MnMigrad(py::module &);
void init_FunctionMinimum(py::module &);

PYBIND11_MODULE(minuit2, m) {
    init_FCNBase(m);
    init_MnUserParameters(m);
    init_MnMigrad(m);
    init_FunctionMinimum(m);
}

Overwriting pyminuit2/pyminuit2.cpp


In [105]:
%%writefile pyminuit2/FCNBase.cpp

#include <pybind11/pybind11.h>
#include <pybind11/stl.h>

#include <Minuit2/FCNBase.h>

namespace py = pybind11;
using namespace ROOT::Minuit2;

class PyFCNBase : public FCNBase {
   public:
     using FCNBase::FCNBase;

     double operator()(const std::vector<double> &v) const override {
         PYBIND11_OVERLOAD_PURE_NAME(double, FCNBase, "__call__", operator(), v);
     }

     double Up() const override {
         PYBIND11_OVERLOAD_PURE(double, FCNBase, Up, );
     }
 };
        
void init_FCNBase(py::module &m) {
    py::class_<FCNBase, PyFCNBase>(m, "FCNBase")
         .def(py::init<>())
         .def("__call__", &FCNBase::operator())
         .def("Up", &FCNBase::Up)
    ;
}

Overwriting pyminuit2/FCNBase.cpp


In [106]:
%%writefile pyminuit2/MnUserParameters.cpp

#include <pybind11/pybind11.h>

#include <pybind11/functional.h>
#include <pybind11/numpy.h>
#include <pybind11/stl.h>

#include <Minuit2/MnUserParameters.h>

#include <string>

namespace py = pybind11;
using namespace pybind11::literals;

using namespace ROOT::Minuit2;

void init_MnUserParameters(py::module &m) {
    py::class_<MnUserParameters>(m, "MnUserParameters")
        .def(py::init<>())
        .def("Add", (bool (MnUserParameters::*)(const std::string &, double)) & MnUserParameters::Add)
        .def("Add", (bool (MnUserParameters::*)(const std::string &, double, double)) & MnUserParameters::Add)
        .def("Add",
             (bool (MnUserParameters::*)(const std::string &, double, double, double, double)) & MnUserParameters::Add)
    ;
}

Overwriting pyminuit2/MnUserParameters.cpp


In [107]:
%%writefile pyminuit2/MnMigrad.cpp

#include <pybind11/pybind11.h>
#include <pybind11/stl.h>

#include <Minuit2/FCNBase.h>
#include <Minuit2/MnMigrad.h>
#include <Minuit2/MnApplication.h>
#include <Minuit2/MnUserParameters.h>
#include <Minuit2/FunctionMinimum.h>

namespace py = pybind11;
using namespace pybind11::literals;
using namespace ROOT::Minuit2;
        
void init_MnMigrad(py::module &m) {
    py::class_<MnApplication>(m, "MnApplication")
        .def("__call__",
             &MnApplication::operator(),
             "Minimize the function, returns a function minimum",
             "maxfcn"_a    = 0,
             "tolerance"_a = 0.1)
    ;
    
    py::class_<MnMigrad, MnApplication>(m, "MnMigrad")
        .def(py::init<const FCNBase &, const MnUserParameters &, unsigned int>(), "fcn"_a, "par"_a, "stra"_a = 1)
    ;
}    

Overwriting pyminuit2/MnMigrad.cpp


In [108]:
%%writefile pyminuit2/FunctionMinimum.cpp

#include <pybind11/pybind11.h>

#include <sstream>

#include <Minuit2/FunctionMinimum.h>
#include <Minuit2/MnPrint.h>

namespace py = pybind11;
using namespace pybind11::literals;

using namespace ROOT::Minuit2;

void init_FunctionMinimum(py::module &m) {
    py::class_<FunctionMinimum>(m, "FunctionMinimum")
        .def("__str__", [](const FunctionMinimum &self) {
            std::stringstream os;
            os << self;
            return os.str();
        })
    ;
}

Overwriting pyminuit2/FunctionMinimum.cpp


In [109]:
!python setup.py build_ext

running build_ext
building 'minuit2' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/FumiliStandardChi2FCN.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/FumiliStandardChi2FCN.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/FumiliBuilder.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/FumiliBuilder.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m 

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/mndscal.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/mndscal.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/VariableMetricBuilder.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/VariableMetricBuilder.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/inc

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/Numerical2PGradientCalculator.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/Numerical2PGradientCalculator.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/hen

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/MnContours.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/MnContours.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinu

gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/.local/include/python3.6m -IMinuit2-6.14.0-Source/inc -I/Users/henryiii/anaconda/include/python3.6m -c Minuit2-6.14.0-Source/src/SinParameterTransformation.cxx -o build/temp.macosx-10.7-x86_64-3.6/Minuit2-6.14.0-Source/src/SinParameterTransformation.o -stdlib=libc++ -mmacosx-version-min=10.7 -DVERSION_INFO="6.14.0" -std=c++14 -fvisibility=hidden
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/henryiii/anaconda/include -arch x86_64 -I/Users/henryiii/anaconda/include -arch x86_64 -DWARNINGMSG -DMATH_NO_PLUGIN_MANAGER -DROOT_Math_VecTypes -I/Users/henryiii/anaconda/include/python3.6m -I/Users/henryiii/

In [2]:
import sys

In [3]:
sys.path.append('build/lib.macosx-10.7-x86_64-3.6')

In [4]:
import minuit2

In [5]:
class SimpleFCN (minuit2.FCNBase):
    def Up(self):
        return 0.5
    def __call__(self, v):
        print("val =", v[0])
        return v[0]**2;


fcn = SimpleFCN()
upar = minuit2.MnUserParameters()
upar.Add("x", 1., 0.1)
migrad = minuit2.MnMigrad(fcn, upar)
min = migrad()
print(min)


val = 1.0
val = 1.001
val = 0.999
val = 1.0005980198587356
val = 0.9994019801412644
val = -8.230083281546285e-11
val = 0.00034526688527999595
val = -0.0003452670498816616
val = -8.230083281546285e-11
val = 0.00034526688527999595
val = -0.0003452670498816616
val = 6.905331121533294e-05
val = -6.905347581699857e-05

Minuit did successfully converge.
# of function calls: 13
minimum function Value: 6.773427082119e-21
minimum edm: 6.773427081817e-21
minimum internal state vector: LAVector parameters:
 -8.230083281546e-11

minimum internal covariance matrix: LASymMatrix parameters:
              1


# ext. ||   Name    ||   type  ||     Value     ||  Error +/- 

   0   ||         x ||  free   || -8.230083281546e-11 ||0.7071067811865





